In [1]:
import sys, os
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)
print("Added to sys.path:/", repo_root)
from fixedincomelib import *
print("Fixed Income Library is loaded.")

Added to sys.path:/ /Users/lunli/Documents/FixedIncomeLib
Fixed Income Library is loaded.


### Create Build Method Collection

In [2]:
### create yield curve build method
bm_list = []
build_method_type = 'YIELD_CURVE'
content_sofr = {
    'TARGET' : 'SOFR-1B',
    'INSTANTANEOUS FORWARD RATE' : 'USD-SOFR-IFR'
}
bm_list.append(qfCreateBuildMethod(build_method_type, content_sofr))
content_ff = {
    'TARGET' : 'FF-1B',
    'INSTANTANEOUS FORWARD RATE' : 'USD-FF-IFR'
}
bm_list.append(qfCreateBuildMethod(build_method_type, content_ff))
build_method_collection = qfCreateModelBuildMethodCollection(bm_list)

### Create Data Collection

In [3]:
data_list = []
df = pd.DataFrame(index=['3M', '6M', '1Y', '5Y'])
df['values'] = [0.03, 0.032, 0.034, 0.035]
data_type = 'Instantaneous Forward Rate'
data_conv = 'USD-SOFR-IFR'
data_list.append(qfCreateData1D(data_type, data_conv, df))
df = pd.DataFrame(index=['3M', '6M', '1Y'])
df['values'] = [0.031, 0.034, 0.036]
data_conv = 'USD-FF-IFR'
data_list.append(qfCreateData1D(data_type, data_conv, df))
data_collection = qfCreateDataCollection(data_list)

### Create Model Yield Curve

In [4]:
value_date = '2025-12-23'
yc_model_1 = qfCreateModel(value_date, 'YIELD_CURVE', data_collection, build_method_collection)
value_date = '2025-09-15'
yc_model_2 = qfCreateModel(value_date, 'YIELD_CURVE', data_collection, build_method_collection)
value_date = '2025-10-27'
yc_model_3 = qfCreateModel(value_date, 'YIELD_CURVE', data_collection, build_method_collection)

In [5]:
display(qfDisplayModelValueDate(yc_model_1))
display(qfDisplayModelType(yc_model_1))

'2025-12-23'

'YIELD_CURVE'

In [6]:
### getter
dc_back = qfGetDataCollectionFromModel(yc_model_1)
bmc_back = qfGetBuildMethodCollection(yc_model_1)
display(dc_back.display())
display(bmc_back.display())

,Data Shape,Data Type,Data Convention
0,DATA1D,Instantaneous Forward Rate,USD-SOFR-IFR
1,DATA1D,Instantaneous Forward Rate,USD-FF-IFR


,Name,Value
0,YIELD_CURVE,SOFR-1B
1,YIELD_CURVE,FF-1B


### Tes Serialization

In [7]:
# serialize
path = 'yc_model.pickle'
qfWriteModelObjectToFile(yc_model_1, path)
# deserialize
model_back : Model = qfReadModelFromFile(path)
# check
display(model_back.value_date)
# house keeping
os.remove(path)

Date(23,12,2025)

### Test Discounting

In [8]:
# sofr discounting
on_index = 'SOFR-1B'
expiry_date = '2026-05-28'
qfDiscountFactor(yc_model_1, on_index, expiry_date)

0.9872599327792846

In [9]:
# sofr df gradient
gradient = []
qfDiscountFactorGradient(yc_model_1, on_index, expiry_date, gradient)
display(gradient)

[array([-0.24343396, -0.17851823, -0.        , -0.        ]),
 array([0., 0., 0.])]

In [10]:
# ff discounting
ff_index = 'FF-1B'
expiry_date = '2026-05-28'
qfDiscountFactor(yc_model_1, ff_index, expiry_date)

0.9868380707465967

In [11]:
# ff df gradient
qfDiscountFactorGradient(yc_model_1, ff_index, expiry_date, gradient)
display(gradient)

[array([-0.24343396, -0.17851823, -0.        , -0.        ]),
 array([-0.24332994, -0.17844195, -0.        ])]

### Test Forward Index Calculation

In [12]:
### not needed for now
vp_type = 'FUNDING INDEX PARAMETER'
content = {'Funding Index' : 'SOFR-1B'}
fi_vp = qfCreateValuationParameters(vp_type, content)
vpc = qfCreateValuationParametersCollection([fi_vp])

In [13]:
### fully fixed
index = 'SOFR-1B'
effective_date = '2025-10-17'
term_or_termination_date = '2025-11-17'
forward_level = qfValueIndexForward(yc_model_1, vpc, index, effective_date, term_or_termination_date)
print(f'forward level is {forward_level}.')
risk_grad = []
qfValueIndexForwardGradient(yc_model_1, vpc, index, effective_date, term_or_termination_date, risk_grad)
display(risk_grad)

forward level is 0.02502516133217782.


[array([0., 0., 0., 0.]), array([0., 0., 0.])]

In [14]:
### nothing has fixed
index = 'SOFR-1B'
effective_date = '2025-10-17'
term_or_termination_date = '2025-11-17'
forward_level = qfValueIndexForward(yc_model_2, vpc, index, effective_date, term_or_termination_date)
print(f'forward level is {forward_level}.')
risk_grad = []
qfValueIndexForwardGradient(yc_model_2, vpc, index, effective_date, term_or_termination_date, risk_grad)
display(risk_grad)

forward level is 0.029626768759457328.


[array([0.98881762, 0.        , 0.        , 0.        ]), array([0., 0., 0.])]

In [15]:
### partially fixed
index = 'SOFR-1B'
effective_date = '2025-10-17'
term_or_termination_date = '2025-11-17'
forward_level = qfValueIndexForward(yc_model_3, vpc, index, effective_date, term_or_termination_date)
print(f'forward level is {forward_level}.')
risk_grad = []
qfValueIndexForwardGradient(yc_model_3, vpc, index, effective_date, term_or_termination_date, risk_grad)
display(risk_grad)

forward level is 0.028142133429624946.


[array([0.66975877, 0.        , 0.        , 0.        ]), array([0., 0., 0.])]